## Project data

The project data comprises 78 locations in Egypt and Isreal, each encompassing several square kilometers. Here we give a quick overview of the data but further down this cell there is more detailed information on each datasource.

For each location there are:

Sentinel-2: There are 9 available bands. These bands are: 2-Blue, 3-Green, 4-Red, 5-RedEdge1, 6-RedEdge2, 7-RedEdge3, 8-NIR, 11-SWIR1, 12-SWIR2. Notice that band 8A-RedEdge is not supplied. These images are not normalised yet, but we will see how to work and visualise them later on.

Sentinel-1: There are 2 available bands which are the VV and VH bands. VV is the mode that transmits vertical waves and receives vertical waves to create the SAR image while VH is the mode that transmits vertical waves and receives horizontal waves. The data has already been despeckled and processed to contain values in  dB .

CopDEM: The Copernicus DEM has 4 channels. The actual elevation in meters is stored in the 4th channel. The first 2 channels store the direction (aspect in sin/cos) the slope is facing while the 3rd channel has the actual slope. All the four channels are in the range [0-1] with channel 4 normalised to the height of Mt. Everest.

Buildings labels: This is the ground truth used for the building density prediction task (building density serves as proxy for population density). The labels are expressed of number of squared meters of building on a given pixel. Values are between 0-100  m2  and for a resolution of 10 m  this reflect the percentage of coverage. This data is a combination of the Google Open Buildings dataset, OSM buildings and manual labeling.

## Sentinel-1: Synthetic Aperture Radar (SAR) data

SENTINEL-1 is an imaging radar mission providing continuous all-weather, day-and-night imagery at C-band at a resolution of about 10m every 12 days. Sentinel-1 is a phase-preserving dual polarisation SAR system. It can transmit a signal in either horizontal (H) or vertical (V) polarisation, and then receive in both H and V polarisations.

SAR sensors are able to detect flooding because flat surfaces reflect (acts as a specular reflector) the signal away from the sensor, decreasing the amount of returned radiation. This causes relatively dark pixels in radar data for water areas which contrast with non-water areas.

Speckle is a general phenomenon in SAR imagery caused by the interaction of the out-of-phase waves reflected from a target resulting in a salt-and-pepper pattern. Presence of speckle in SAR images degrades the interpretability of the land features in the data. Speckle removal is necessary for quantitative, analysis and there are various filters but there exists a tradeoff between speckle removal and resolution. The images we use have already been despeckled and processed to dB values.

## Sentinel-2: Multispectral data

SENTINEL-2 is a wide-swath, high-resolution, multi-spectral imaging mission, supporting Copernicus Land Monitoring studies, including the monitoring of vegetation, soil and water cover, as well as observation of inland waterways and coastal areas. Most importantly for us is the MultiSpectral Instrument (MSI), which samples 13 spectral bands: four bands at 10 metres, six bands at 20 metres and three bands at 60 metres spatial resolution. These include the Blue, Green and Red bands which correspond to how humans see the world but also Near Infrared (NIR) and Short-wave Infrared (SWIR) which can be used to monitor vegetation, geological features and much more.

Sentinel-2 images are very valuable for population density estimations and the mapping of vulnerable areas. The infrared bands are effective to differentiate between different types of land use while the Red band is used for mapping man-made structures. The high resolution RGB images can be used to detect schools, hospitals and other vulnerable areas.

## Copernicus DEM: Digital Elevation Model data

The Copernicus DEM is a Digital Surface Model (DSM) that represents the surface of the Earth including buildings, infrastructure and vegetation. The Copernicus DEM provides digital elevation maps for Egypt at a resolution of 30m. Data to create the DEM were acquired through the TanDEM-X mission.

As one can image, DEM’s are often used for flood prediction. DEMs can be used to create flood inundation maps, which show how water will spread across the landscape during a flood event. By combining a DEM with hydrological modeling, it is possible to simulate flood events and create maps that show which areas will be affected by flooding and to what extent. DEMs can also be used to identify natural features that can affect flooding, such as ridges, valleys, and drainage basins.


## Libraries

In [8]:
# Standard library imports
import os
import zipfile
from pathlib import Path

# Third-party imports
import gdown

## Constants

In [9]:
download_url = "https://drive.google.com/uc?id=1N-vOm_rBPXM4CbXTdHhKfjYSeJd93sL0"
output_folder = os.path.abspath(os.path.join(os.getcwd(), '../data/CCAI_FLOODS_DATA'))

# Sentinel-1 and Sentinel-2 data
s1_data = 'https://drive.google.com/uc?id=1LrakuW_RVA8KVsls4NyesnG4pFmxu7eS'
s2_data = 'https://drive.google.com/uc?id=1lrvPBJXHYmA6Xcd9bklNRHjcP9e_zQvQ'

## Download Data

In [ ]:
def download_from_google_drive(url: str, filename: str, output_path: Path = None, extract: bool = False):
    """
    Downloads a file from Google Drive and unzips it to the specified output path.

    Args:
        url (str): The Google Drive URL to download the file from.
        filename (str): The name of the file to save.
        output_path (Path, optional): The directory where the file will be saved. Defaults to None.
        extract (bool, optional): Whether to extract the file if it is a zip file. Defaults to False.
    """
    # If output_path is provided, ensure it exists
    if output_path:
        output_path.mkdir(parents=True, exist_ok=True)

    # Download the file
    print(f"Initiating download from Google Drive...")
    gdown.download(url, filename, quiet=False)
    print(f"Download completed")

    # If extract is True and output_path is provided, extract the file to that path
    if extract and output_path:
        print(f"Unzipping {filename}...")
        # Unzip the file
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(output_path)
        print(f"Unzipping completed")

        # Remove the zip file after extraction
        os.remove(filename)

In [11]:
# Download dataset and extract it
download_from_google_drive(download_url, 'CCAI_FLOODS_DATA.zip', Path(output_folder), extract=True)

Downloading...
From (original): https://drive.google.com/uc?id=1N-vOm_rBPXM4CbXTdHhKfjYSeJd93sL0
From (redirected): https://drive.google.com/uc?id=1N-vOm_rBPXM4CbXTdHhKfjYSeJd93sL0&confirm=t&uuid=91369760-07ec-4a73-98e9-1cf8d79437d7
To: c:\Users\dango\OneDrive - UWE Bristol\projects\flood-monitoring\notebooks\CCAI_FLOODS_DATA.zip
100%|██████████| 395M/395M [00:14<00:00, 28.1MB/s] 


Download completed
Unzipping CCAI_FLOODS_DATA.zip...
Unzipping completed


## Download Sentinel-1 and Sentinel-2 Data

In [12]:
# Constants for Sentinel-1 and Sentinel-2 data
output_folder = os.path.abspath(os.path.join(os.getcwd(), '../data'))
s1_folder = os.path.join(output_folder, 'S1.tif')
s2_folder = os.path.join(output_folder, 'S2.tif')

### Sentinel-1

In [13]:
# Download Sentinel-1 data
download_from_google_drive(
  s1_data,
  filename=s1_folder,
)

Downloading...
From (original): https://drive.google.com/uc?id=1LrakuW_RVA8KVsls4NyesnG4pFmxu7eS
From (redirected): https://drive.google.com/uc?id=1LrakuW_RVA8KVsls4NyesnG4pFmxu7eS&confirm=t&uuid=3ff992e7-2825-400a-855c-6731540fa665
To: c:\Users\dango\OneDrive - UWE Bristol\projects\flood-monitoring\data\S1.tif
100%|██████████| 819M/819M [00:29<00:00, 27.4MB/s] 

Download completed


### Sentinel-2

In [14]:
# Download Sentinel-2 data
download_from_google_drive(
  s2_data,
  filename=s2_folder
)

Downloading...
From (original): https://drive.google.com/uc?id=1lrvPBJXHYmA6Xcd9bklNRHjcP9e_zQvQ
From (redirected): https://drive.google.com/uc?id=1lrvPBJXHYmA6Xcd9bklNRHjcP9e_zQvQ&confirm=t&uuid=4b462c5e-c2dc-4493-99b1-db3224c528ad
To: c:\Users\dango\OneDrive - UWE Bristol\projects\flood-monitoring\data\S2.tif
100%|██████████| 1.53G/1.53G [00:55<00:00, 27.4MB/s]

Download completed
